In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torchvision import datasets, transforms
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np
import sys
import copy
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
import pandas as pd
import pickle



# 하이퍼파라미터 설정
batch_size = 100
epochs = 10
lr = 0.01
PCA_dim = 8
CLS_num = 2



with open('./data/data.pkl','rb') as file:
    data = pickle.load(file)
X = data['X']
y = data['Y']



x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)



def Fit_to_quantum(X,PCA_dim):
    pca = PCA(n_components=PCA_dim)
    X_pca = pca.fit_transform(X)
    return X_pca
    


# 정규화 (표준 스케일러 사용)
#x_train_pca = Fit_to_quantum(x_train,PCA_dim)
#x_test_pca = Fit_to_quantum(x_test,PCA_dim)

# PyTorch Tensor로 변환
x_train_pca, y_train = torch.tensor(x_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long)
x_test_pca, y_test = torch.tensor(x_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long)


class Feature_data_loader(Dataset):
    def __init__(self,x_train,y_train):
        self.feature1 = x_train
        temp = copy.deepcopy(x_train)
        shuffle = torch.randperm(len(temp))
        self.feature2 = temp[shuffle]
        
        self.y1 = y_train
        temp_y = copy.deepcopy(y_train)
        self.y2 = temp_y[shuffle]
        
    def __len__(self):
        return len(self.feature1)
    def __getitem__(self,idx):
        input1 = self.feature1[idx]
        input2 = self.feature2[idx]
        if self.y1[idx] == self.y2[idx]:
            label = torch.tensor(1.).float()
        else:
            label = torch.tensor(0.).float()
        return [input1,input2],label


# DataLoader 생성


train_loader = DataLoader(TensorDataset(x_train_pca, y_train.float()), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(TensorDataset(x_test_pca, y_test.float()), batch_size=batch_size, shuffle=False)


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from collections import OrderedDict
import math
from kan import KAN
from functions.training import Early_stop_train




class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.cls_layer_1 = nn.Linear(PCA_dim,PCA_dim*PCA_dim)
        self.cls_layer_2 = nn.Linear(PCA_dim*PCA_dim,PCA_dim*2-1)
        self.output_layer = nn.Linear(2*PCA_dim-1,1)
    def forward(self, x):
        x = self.cls_layer_1(x)
        x = nn.ReLU()(x)
        x = self.cls_layer_2(x)
        x = nn.ReLU()(x)
        output = self.output_layer(x)
        output = nn.Sigmoid()(output)
        output = torch.squeeze(output)
        return output
# 모델, 손실 함수, 최적화 설정



model = Model(); criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)




train_process = Early_stop_train(model, optimizer, criterion)
train_process.train_model(train_loader,test_loader,epochs=5000,res=15)

_,acc = train_process.test(test_loader)
print(f"Test Accuracy: {acc:.2f}")




Test set: Average loss: 2.1512, Accuracy: 117.9999977350235/300 (39%)
Epoch 1 Loss 0.711127 acc : 0.441429 stop count : 0
Test set: Average loss: 2.1320, Accuracy: 124.00000095367432/300 (41%)
Epoch 2 Loss 0.704312 acc : 0.471429 stop count : 0
Test set: Average loss: 2.1136, Accuracy: 128.99999916553497/300 (43%)
Epoch 3 Loss 0.697845 acc : 0.488571 stop count : 0
Test set: Average loss: 2.0958, Accuracy: 137.00000047683716/300 (46%)
Epoch 4 Loss 0.691487 acc : 0.507143 stop count : 0
Test set: Average loss: 2.0785, Accuracy: 141.00000262260437/300 (47%)
Epoch 5 Loss 0.685311 acc : 0.537143 stop count : 0
Test set: Average loss: 2.0615, Accuracy: 143.99999976158142/300 (48%)
Epoch 6 Loss 0.679120 acc : 0.570000 stop count : 0
Test set: Average loss: 2.0453, Accuracy: 153.99999618530273/300 (51%)
Epoch 7 Loss 0.673229 acc : 0.591429 stop count : 0
Test set: Average loss: 2.0291, Accuracy: 167.00000166893005/300 (56%)
Epoch 8 Loss 0.667553 acc : 0.620000 stop count : 0
Test set: Averag

In [3]:
from kan import KAN
from functions.training import Early_stop_train_KAN
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.KAN = KAN([PCA_dim,PCA_dim*2-1,1],grid=5)
    def forward(self, x):
        output = self.KAN(x)
        output = nn.Sigmoid()(output)
        output = torch.squeeze(output)
        return output


model = Model(); criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)




train_process = Early_stop_train_KAN(model, optimizer, criterion)
train_process.train_model(train_loader,test_loader,epochs=50,res=15,lamb=0.005)

_,acc = train_process.test(test_loader)
print(f"Test Accuracy: {acc:.2f}")


Test set: Average loss: 2.4758, Accuracy: 152.99999713897705/300 (51%)
Epoch 1 Loss 0.817065 acc : 0.657143 reg : 0.176591 stop count : 0
Test set: Average loss: 1.5876, Accuracy: 218.00000071525574/300 (73%)
Epoch 2 Loss 0.630989 acc : 0.805714 reg : 0.173415 stop count : 0
Test set: Average loss: 1.2217, Accuracy: 249.00000095367432/300 (83%)
Epoch 3 Loss 0.571237 acc : 0.832857 reg : 0.179782 stop count : 0
Test set: Average loss: 1.1076, Accuracy: 255.0000011920929/300 (85%)
Epoch 4 Loss 0.537337 acc : 0.844286 reg : 0.181966 stop count : 0
Test set: Average loss: 1.0830, Accuracy: 250.9999930858612/300 (84%)
Epoch 5 Loss 0.514785 acc : 0.848571 reg : 0.178449 stop count : 0
Test set: Average loss: 1.0206, Accuracy: 256.00000619888306/300 (85%)
Epoch 6 Loss 0.489433 acc : 0.860000 reg : 0.175133 stop count : 0
Test set: Average loss: 0.9901, Accuracy: 256.00000619888306/300 (85%)
Epoch 7 Loss 0.460913 acc : 0.885714 reg : 0.179059 stop count : 0
Test set: Average loss: 0.9521, Acc

In [4]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

In [20]:
# SVM 모델 생성 및 훈련
svm_model = SVC(kernel='poly', C=1.0, random_state=42)
svm_model.fit(x_train, y_train)

# Validation set으로 예측
y_val_pred = svm_model.predict(x_test)

# 평가
accuracy = accuracy_score(y_test, y_val_pred)
report = classification_report(y_test, y_val_pred)

print(f'Validation Accuracy: {accuracy}')
print('Classification Report:')
print(report)

Validation Accuracy: 0.8566666666666667
Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.77      0.84       147
           1       0.81      0.94      0.87       153

    accuracy                           0.86       300
   macro avg       0.87      0.85      0.86       300
weighted avg       0.87      0.86      0.86       300

